In [1]:
from  Dataset import *

In [2]:
from Config import *

In [3]:
dataset = Dataset()

> Dataset Class Initialized
>> loading Data: BTCUSDT
>> Data Loaded


In [4]:
train, test = dataset.ProcessData()

In [5]:
train

,Low,High,Open,Close,Volume,RSI,EMA,SMA,DCL_10_15,DCM_10_15,...,Low_pct_change,High_pct_change,Open_pct_change,Close_pct_change,EMA_pct_change,SMA_pct_change,DCL_10_15_pct_change,DCM_10_15_pct_change,buy,sell
time,,,,,,,,,,,,,,,,,,,,,
1590949860000,0.107939,0.107586,0.107577,0.107774,0.004642,0.434708,0.107512,0.107693,0.108168,0.107578,...,0.420917,0.377896,0.475769,0.491574,0.453611,0.436419,0.499811,0.408590,0,0
1590949920000,0.107903,0.107516,0.107587,0.107738,0.003763,0.410708,0.107499,0.107680,0.108168,0.107578,...,0.417202,0.373568,0.478676,0.489502,0.453074,0.434733,0.499811,0.408590,0,0
1590949980000,0.107699,0.107464,0.107550,0.107519,0.010451,0.303215,0.107470,0.107648,0.107997,0.107493,...,0.411179,0.374244,0.476610,0.481452,0.450362,0.431308,0.492289,0.402143,0,0
1590950040000,0.107700,0.107321,0.107333,0.107564,0.006700,0.341294,0.107435,0.107622,0.107997,0.107493,...,0.418482,0.370886,0.468670,0.493107,0.449167,0.432381,0.499811,0.408590,0,0
1590950100000,0.107697,0.107313,0.107384,0.107544,0.006557,0.332614,0.107405,0.107600,0.107995,0.107479,...,0.418361,0.375859,0.480476,0.490244,0.450125,0.433048,0.499705,0.407583,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1609439100000,0.418519,0.417805,0.417842,0.418155,0.009330,0.669541,0.418427,0.418418,0.418347,0.418296,...,0.416465,0.376156,0.474639,0.493915,0.462210,0.441970,0.501832,0.410319,0,0
1609439160000,0.418846,0.417849,0.418025,0.418158,0.005514,0.669812,0.418545,0.418517,0.418451,0.418348,...,0.422326,0.376698,0.480868,0.491169,0.462400,0.442835,0.501328,0.409889,0,0
1609439220000,0.418884,0.417859,0.418036,0.418109,0.008732,0.660415,0.418648,0.418607,0.418451,0.418348,...,0.418915,0.376274,0.478384,0.490396,0.461490,0.442344,0.499811,0.408590,0,0


In [6]:
def make_seq(df):
    seq_data = []
    deq = deque(maxlen=LOOK_BACK_LEN)
    for index, row in df.iterrows():
        deq.append(row[:len(df.columns)-2].values)
#         print(row[:len(df.columns)-2].values)
        if len(deq) == LOOK_BACK_LEN:
#             print(row[len(df.columns)-2:].values)
            seq_data.append([ np.array(list(deq)),  np.array(row[len(df.columns)-2:].values) ])
    return seq_data

In [7]:
def split_xy(arr):
    x = []
    y = []

    for seq, target in train_seq:
        x.append(seq)
        y.append(target)

    x = np.array(x)
    y = np.array(y)
    return x,y

In [8]:
train_seq  = make_seq(train)


KeyboardInterrupt: 

In [ ]:
print(train_seq[0])

In [ ]:
print(x.shape)
print(y.shape)

In [5]:
from keras.preprocessing.sequence import TimeseriesGenerator
from tensorflow.keras.layers import Dense, Dropout, LSTM, BatchNormalization
from tensorflow.keras import Sequential
import tensorflow as tf

In [6]:
ts_generator = TimeseriesGenerator(train[[*train.columns[:len(train.columns)-2]]].values, train[[*train.columns[-1:]]].values, length=LOOK_BACK_LEN, sampling_rate=1, batch_size=1)

In [7]:
model = Sequential()
model.add(LSTM(128, input_shape=(360,19), return_sequences=True))
model.add(Dropout(0.2))
model.add(BatchNormalization())  #normalizes activation outputs, same reason you want to normalize your input data.

model.add(LSTM(128, return_sequences=True))
model.add(Dropout(0.1))
model.add(BatchNormalization())

model.add(LSTM(128))
model.add(Dropout(0.2))
model.add(BatchNormalization())

model.add(Dense(32, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(2, activation='softmax'))

In [8]:
opt = tf.keras.optimizers.Adam(lr=0.001, decay=1e-6)

# Compile model
model.compile(
    loss='categorical_crossentropy',
    optimizer=opt,
    metrics=['accuracy']
)

C:\ProgramData\Anaconda3\lib\site-packages\keras\optimizer_v2\optimizer_v2.py:355: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


In [ ]:
model.fit(
    ts_generator,
    epochs=20,
)


Epoch 1/20
